In [ ]:
# The preliminary notebook for testing/developing wrsamp. Will delete before merging. 

In [30]:
from collections import OrderedDict
fields = OrderedDict([('height', 170), ('weight', 70), ('age', 20)])

for f in fields:
    print(f)

height
weight
age


In [35]:
from collections import OrderedDict
fields = OrderedDict({'height': 170, 'weight': 70, 'age': 20})

for f in fields:
    print(f)

age
height
weight


In [37]:
dependencies = OrderedDict([('signame', 'checksum'),
                                ('blocksize', 'checksum'),
                                ('checksum', 'initvalue'),
                                ('initvalue', 'adczero'),
                                ('adczero', 'adcres'),
                                ('adcres', 'adcgain'),
                                ('units', 'adcgain'),
                                ('baseline', 'adcgain'),
                                ('adcgain', 'fmt'),
                                ('byteoffset', 'fmt'),
                                ('skew', 'fmt'),
                                ('sampsperframe', 'fmt')
                               ])

for d in dependencies:
    print(d)
    print(dependencies[d])

signame
checksum
blocksize
checksum
checksum
initvalue
initvalue
adczero
adczero
adcres
adcres
adcgain
units
adcgain
baseline
adcgain
adcgain
fmt
byteoffset
fmt
skew
fmt
sampsperframe
fmt


In [7]:
fields = {'height': 170, 'weight': 70, 'age': 20}

for i in reversed(list(fields)):
    print(i)

weight
height
age


SyntaxError: invalid syntax (<ipython-input-17-8c8a4646591e>, line 1)

In [48]:
import numpy as np
a=np.zeros([10,3])
m, n = np.shape(a)


In [4]:
WFDBfields = ['height', 'weight', 'age']

final = dict.fromkeys(WFDBfields, [])
final

{'age': [], 'height': [], 'weight': []}

In [18]:
help(dict.fromkeys)

Help on built-in function fromkeys:

fromkeys(iterable, value=None, /) method of builtins.type instance
    Returns a new dict with keys from iterable and values equal to value.



In [20]:
for i in final:
    print(i)

age
weight
height


In [49]:
if np.shape(a) == (10, 3):
    print('he')

he


In [5]:
for i in reversed(WFDBfields):
    print(i)

age
weight
height
